In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# load MATLAB files
from scipy.io import loadmat

In [2]:
data = loadmat('data/ex4data1.mat')
X = data['X'];
ones = np.ones((X.shape[0],1))
X = np.c_[ones,X]
y = data['y']
print("Shape of X (with intercept) :",X.shape)
print("Shape of y ",y.shape)
weights = loadmat('data/ex4weights.mat')
theta1 = weights['Theta1']
theta2 = weights['Theta2']
print("\n")
print("Shape of Theta 1:",theta1.shape)
print("Shape of Theta 2:",theta2.shape)

Shape of X (with intercept) : (5000, 401)
Shape of y  (5000, 1)


Shape of Theta 1: (25, 401)
Shape of Theta 2: (10, 26)


In [24]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [25]:
def sigmoid_gradient(z):
    return sigmoid(z)*(1-sigmoid(z))

In [26]:
# sigmoid(0)

In [51]:
def nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda):
    Theta1 = nn_params[0:(input_layer_size+1)*(hidden_layer_size)].reshape(hidden_layer_size,input_layer_size+1)
    Theta2 = nn_params[(input_layer_size+1)*(hidden_layer_size):].reshape(num_labels,hidden_layer_size+1)
    
    y_matrix = pd.get_dummies(y.ravel()).as_matrix()
    
#     forward propogation
    z2 = np.dot(X,Theta1.T)
    a2 = np.c_[np.ones((z2.shape[0],1)),sigmoid(z2)]
    z3 = np.dot(a2,Theta2.T)
    a3 = sigmoid(z3)
     
    m = a3.shape[0]
    
    J = -1*(1/m)*np.sum((y_matrix*np.log(a3))+((1-y_matrix)*np.log(1-a3)))+(Lambda/(2*m))*(np.sum(np.square(theta1[:,1:])) + np.sum(np.square(theta2[:,1:])))
#     print(J)
  
#     print(a3.shape)
#     print(Theta2.shape)
#     print(z2.shape)
    delta3 = a3 - y_matrix
    delta2 = np.multiply(np.dot(delta3,Theta2[:,1:]),sigmoid_gradient(z2))
    
    D2 = np.dot(delta3.T,a2)
    D1 = np.dot(delta2.T,X)
    
    theta1_reg=np.c_[np.ones((Theta1.shape[0],1)),Theta1[:,1:]]
    theta2_reg=np.c_[np.ones((Theta2.shape[0],1)),Theta2[:,1:]]
    
    theta1_grad = (1/m)*D1+(1/m)*(theta1_reg*Lambda)
    theta2_grad = (1/m)*D2+(1/m)*(theta2_reg*Lambda)
    
    grad = np.r_[theta1_grad.ravel(),theta2_grad.ravel()]
    
    return (J,grad)
    

In [54]:
input_layer_size = 400
hidden_layer_size = 25
num_labels = 10
Lambda = 0
nn_params = np.hstack((theta1.ravel(),theta2.ravel()))
#print(nn_params.shape)
nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda)
Lambda = 1
nnCostFunction(nn_params, input_layer_size, hidden_layer_size, num_labels, X, y, Lambda)

(0.38376985909092365,
 array([  2.61871277e-04,  -2.11248326e-12,   4.38829369e-13, ...,
          4.70513145e-05,  -5.01718610e-04,   5.07825789e-04]))